In [30]:
%%time
import awswrangler as wr
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

role = get_execution_role()
region = boto3.Session().region_name

containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

bucket = 'ads508team7'
prefix = 'jump_start'

CPU times: user 68.2 ms, sys: 4.46 ms, total: 72.6 ms
Wall time: 144 ms


In [31]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[region],role, instance_count=1, instance_type='ml.m5.xlarge',output_path='s3://{}/{}/output'.format(bucket, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [32]:
%%time
xgb.fit({'train': s3_input_train})

2022-04-18 06:09:09 Starting - Starting the training job...
2022-04-18 06:09:33 Starting - Preparing the instances for trainingProfilerReport-1650262149: InProgress
......
2022-04-18 06:10:35 Downloading - Downloading input data...
2022-04-18 06:11:06 Training - Downloading the training image.....Arguments: train
[2022-04-18:06:11:49:INFO] Running standalone xgboost training.
[2022-04-18:06:11:49:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-04-18:06:11:49:INFO] File size need to be processed in the node: 25.48mb. Available memory size in the node: 8147.4mb
[2022-04-18:06:11:49:INFO] Determined delimiter of CSV input is ','
[06:11:49] S3DistributionType set as FullyReplicated
[06:11:50] 509880x25 matrix with 12747000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[06:11:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 42 extra nodes, 8 pruned nodes, max_depth=5
[0]#011train-error:0.406904
[06:11:51] src/tree/updater_prune

In [33]:
%%time
xgb_predictor = xgb.deploy(initial_instance_count=5,instance_type='ml.m5.xlarge')

------!CPU times: user 114 ms, sys: 7.97 ms, total: 122 ms
Wall time: 3min 1s


In [54]:
from IPython import display
from sagemaker.serializers import CSVSerializer
xgb_predictor.serializer = CSVSerializer()

def get_prediction_row (features):
    values = xgb_predictor.predict(features).decode('utf-8')
#     output = [1 if float(value) >= 0.5 else 0 for value in values.split(',')]
    output = [f'{float(value):0.3g}' for value in values.split(',')]
    return pd.DataFrame(output, columns = ['Pred'])


    

def get_prediction_per_all(df):
    step = 10000
    prediction_set = pd.DataFrame()
    for i in range(0,len(df),step):
        feature_sets = df.loc[i:i + (step *1)-1, df.columns!='Target']
        input_values = feature_sets.values.tolist()
        output_predictions = get_prediction_row(input_values)
        prediction_set = pd.concat([prediction_set, output_predictions],ignore_index=True)
        display.clear_output(wait=True)
        display.display(i)
    return prediction_set

In [103]:
prediction_set_test = get_prediction_per_all(df_test)  
df_pred_test = df_test.join(prediction_set_test)
wr.s3.to_csv(df_pred_test, 's3://ads508team7/jump_start/pred_test/data.csv', index = False)

prediction_set_valid = get_prediction_per_all(df_valid)   
df_pred_valid = df_test.join(prediction_set_valid)
wr.s3.to_csv(df_pred_valid, 's3://ads508team7/jump_start/pred_valid/data.csv', index = False)

500000

{'paths': ['s3://ads508team7/jump_start/pred_valid/data.csv'],
 'partitions_values': {}}